In [13]:
from open3d import *
import numpy as np
import re
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from PIL import Image

In [14]:
focalLengthX = 509.9149156563371
focalLengthY = 507.9923678046019
centerX = 336.435697832244
centerY = 245.4809047393366
scalingFactor = 0.0012

"""
[509.9149156563371, 0, 336.435697832244;
0, 507.9923678046019, 245.4809047393366;
0, 0, 1]
"""

"""
focalLengthX = 1076.74064739
focalLengthY = 1075.17825536
centerX = 366.98264967
centerY = 271.59181836
scalingFactor = 0.1
"""
def generate_pointcloud(rgb_file,depth_file,ply_file):
    """
    Generate a colored point cloud in PLY format from a color and a depth image.
    
    Input:
    rgb_file -- filename of color image
    depth_file -- filename of depth image
    ply_file -- filename of ply file
    
    """
    rgb = Image.open(rgb_file)
    depth = Image.open(depth_file)
    
    if rgb.size != depth.size:
        raise Exception("Color and depth image do not have the same resolution.")
    if rgb.mode != "RGB":
        raise Exception("Color image is not in RGB format")
    if depth.mode != "I":
        raise Exception("Depth image is not in intensity format")


    points = []    
    for v in range(rgb.size[1]):
        for u in range(rgb.size[0]):
            color = rgb.getpixel((u,v))
            Z = depth.getpixel((u,v)) * scalingFactor
            if Z==0: continue
            X = (u - centerX) * Z / focalLengthX
            Y = (v - centerY) * Z / focalLengthY
            points.append("%f %f %f %d %d %d 0\n"%(X,Y,Z,color[0],color[1],color[2]))
    file = open(ply_file,"w")
    file.write('''ply
format ascii 1.0
element vertex %d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
property uchar alpha
end_header
%s
'''%(len(points),"".join(points)))
    file.close()


In [15]:
color_raw_path = "save_files/scene/rgb/rgb_1.png"
depth_raw_path = "save_files/scene/depth/depth_1.png"

generate_pointcloud(color_raw_path,depth_raw_path,"test1.ply")

In [16]:
pcd = read_point_cloud("scene_6.ply")
draw_geometries([pcd])

In [12]:
import glob
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]

rgd_list = glob.glob('save_files/rgb/*.png')
rgd_list.sort(key=natural_keys) 

depth_list =glob.glob('save_files/depth/*.png')
depth_list.sort(key=natural_keys) 

In [7]:
import os, sys
 
header = "# .PCD v.7 - Point Cloud Data file format\n\
FIELDS x y z\n\
SIZE 4 4 4\n\
TYPE F F F\n\
COUNT 1 1 1\n\
WIDTH XXXX\n\
HEIGHT 1\n\
VIEWPOINT 0 0 0 1 0 0 0\n\
POINTS XXXX\n\
DATA ascii"

def convertPLYToPCD(mesh_file, pcd_file):
    input = open(mesh_file)
    out = pcd_file
    output = open(out, 'w')
    write_points = False
    points_counter = 0
    nr_points = 0
    for s in input.readlines():
        if s.find("element vertex") != -1:
            nr_points = int(s.split(" ")[2].rstrip().lstrip())
            new_header = header.replace("XXXX", str(nr_points))
            output.write(new_header)
            output.write("\n")
        if s.find("end_header") != -1:
            write_points = True
            continue
        if write_points and points_counter < nr_points:
            points_counter = points_counter + 1
            output.write(" ".join(s.split(" ", 3)[:3]))
            #output.write("\n")
    input.close()
    output.close()

In [46]:
i=0
for color_raw_path,depth_raw_path in zip(rgd_list,depth_list):
    name_ply = "save_files/point_cloud/ply/pc_" + str(i) + ".ply"
    name_pcd = "save_files/point_cloud/pcd/pc_" + str(i) + ".pcd"
    generate_pointcloud(color_raw_path,depth_raw_path,name_ply)
    convertPLYToPCD(name_ply, name_pcd)
    #pcd = read_point_cloud(name_ply)
    #draw_geometries([pcd])
    i+=1

FileNotFoundError: [Errno 2] No such file or directory: 'save_files/rgb/rgb_11.png'